Un  sitio  de  E books  tiene  información  sobre  los  reviews  que  los  usuarios hacen  de  sus  libros  en  un  DataFrame  con  formato  (user_id,  book_id,  rating, timestamp). 

Por otro lado tenemos información en otro DataFrame que bajamos de  GoodReads:  (book_id,  book_name,  avg_rating).  Podemos  suponer  que  los Ids de los libros son compatibles. 

Se pide usar Python Pandas para:

1. Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame).
2. Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads.

[Link](https://piazza.com/class_profile/get_resource/jkr2voxi1yw4wt/jkr2vqsptt94yl)

In [2]:
# Set-up y vista rápida de los dos datasets sintéticos
import pandas as pd

ebooks_df = pd.read_csv('../data/2017C1_1_ebooks.csv')
display(ebooks_df)

goodreads_df = pd.read_csv('../data/2017C1_1_goodreads.csv')
display(goodreads_df)

,user_id,book_id,rating,timestamp
0,1,1,4,2017-05-28 23:38:05
1,2,1,5,2017-05-28 23:38:06
2,1,2,1,2017-05-29 23:30:05
3,2,4,3,2017-05-29 23:32:05
4,3,3,2,2017-05-29 22:12:12
5,4,3,1,2017-05-30 22:12:15
6,1,5,2,2017-05-30 22:12:20
7,5,2,3,2017-05-31 23:38:05
8,4,2,4,2017-05-31 23:38:08
9,6,6,4,2017-05-31 23:38:20


,book_id,book_name,avg_rating
0,1,To Kill a MockingBird,5
1,2,Don Quixote,4
2,3,Lazarillo de Tormes,4
3,4,1984,5
4,5,Fifty Shades of Grey,1
5,6,Los hombres que no amaban a las mujeres,4
6,7,Corazon de tinta,5
7,8,Harry Potter,5


**1. Obtener un DataFrame que indique el TOP5 de Ebooks en el sitio de Ebooks. (Para este punto se puede ignorar el segundo DataFrame)**

Los pasos a seguir serían los siguientes:

1. Agrupamos por libro
2. Calculamos el score de cada libro en base a los ratings
3. Tomamos el top5

In [8]:
def score(s):
    return s.mean()

ebooks_df.groupby('book_id').agg({'rating':score}).nlargest(5, 'rating')

,rating
book_id,
1,4.5
6,4.0
7,4.0
8,4.0
4,3.5


Notar que en el paso anterior usamos el promedio como función de score, pero esto trae un problema: Si consideramos libros con pocas reviews muy positivas, no es representativo realmente. Es una consecuencia de la [ecuación mas peligrosa de la historia](http://www-stat.wharton.upenn.edu/~hwainer/Readings/Most%20Dangerous%20eqn.pdf). Un approach más inteligente sería considerar la cantidad de reviews también.

**2. Obtener un DataFrame que indique qué libros tienen una diferencia de rating promedio mayor al 20% entre el sitio de Ebooks y GoodReads**

Acá, vamos a tener que juntar los dos dataframes, calcular el rating promedio del primer dataframe, calcular la el porcentaje de ese rating promedio sobre el promedio de goodreads, filtrar donde ese número sea mayor al 20% y devolver todos los nombres.

Entonces:


1. Calcular el rating del primer dataset
1. Hacer un inner join entre ambos datasets, usando la columna de book_id para juntar.
1. Devolver los nombres donde la diferencia sea mayor al 20%

In [10]:
ebooks_df_mean = ebooks_df.groupby('book_id').agg({'rating':score})

In [13]:
df_total = goodreads_df.join(ebooks_df_mean, on='book_id',how='inner')
df_total

,book_id,book_name,avg_rating,rating
0,1,To Kill a MockingBird,5,4.500000
1,2,Don Quixote,4,2.666667
2,3,Lazarillo de Tormes,4,1.500000
3,4,1984,5,3.500000
4,5,Fifty Shades of Grey,1,2.000000
5,6,Los hombres que no amaban a las mujeres,4,4.000000
6,7,Corazon de tinta,5,4.000000
7,8,Harry Potter,5,4.000000


In [16]:
df_total[(df_total.rating/df_total.avg_rating - 1.0).abs() > 0.2].book_name.tolist()

['Don Quixote', 'Lazarillo de Tormes', '1984', 'Fifty Shades of Grey']